# Info from the notebooks on SSJ

SSJ has several subclasses of the abstract **Block** class.
- `SimpleBlock` (`sj.@simple`) for explicit aggregate equilibrium conditions
- `SolvedBlock` (`@sj.solved`) for implicit aggregate equilibrium conditions
- `HetBlock` (`@sj.het`) for collective actions of heterogeneous agents
- `CombinedBlock` for combination of multiple blocks

They are followed with by a fct definition.

Notation conventions are as follows: 
   - `Y` refers to $Y_t$
   - `Y(-k)` refers to $Y_{t-k}$
   - `Y(k)` refers to $\mathbb{E}_t[Y_{t+k}]$
   - lag variables cannot be inputted but you can cal them from another bock (fct) that returns the lag of a var given the var.

# Understanding HH block


In [31]:
import numpy as np
import sequence_jacobian as sj # sequence-space Jacobian toolkit
from scipy import linalg

In [32]:
# Income process from Kaplan Moll Violante 2018
# load KMV transition matrix, convert to discrete time, and fix rounding so rows sum to 1
Pi_e = linalg.expm(np.loadtxt('inputs/kmv_process/ymarkov_combined.txt'))
Pi_e /= np.sum(Pi_e, axis=1)[:, np.newaxis]


`Pi_e` = Transition Matrix for Income (the probabilities of transitioning between different income states). The rows of Pi_e correspond to the current state, and the columns correspond to the next state.

`np.loadtxt`: Reads the text doc in the folders inputs/kmv_process/ that shows the income transition matrix in discrete time. 

`linalg.expm` = Compute the matrix exponential of an array to convert it from a continuous-time to a discrete-time Markov process.

`np.sum(Pi_e, axis=1)`= compute the sum of each row in the matrix => outputs array of shape (n,) with each element inside on n the sum of a row (n rows).

`[:, np.newaxis]` changes its shape to (n, 1).

`/=` = division of each element of `Pi_e` => normalizing, and each row will sum one. WHY? each value of a row is a state you can acces in the next period and the sum of the probabilities of the different states must be one. 

In [33]:
pi_e = sj.utilities.discretize.stationary(Pi_e) # need to find the official function definition
e_grid_short = np.exp(np.loadtxt('inputs/kmv_process/ygrid_combined.txt'))
# the exponential of a grid of logarithmic income levels => actual income levels
e_grid_short /= e_grid_short @ pi_e # normalize so that the mass of agents is always 1?
n_e = len(pi_e) 

`sj.utilities.discretize.stationary` = calculates the stationary distribution of the Markov chain (LR/equilibrium distribution of agents across income states)

`pi_e`= probably (?) the mass of agents in each income state at equilibrium.

In [34]:
print(e_grid_short.size)
print(n_e)
print(pi_e)
# there are possible 33 income states

33
33
[2.63255204e-04 7.31719227e-04 1.53413313e-03 8.94710875e-03
 2.55626630e-03 3.27734757e-03 2.48685358e-02 1.10627972e-02
 3.27734757e-03 5.21397327e-02 2.55626630e-03 2.63255204e-04
 8.68784064e-02 1.53413313e-03 1.11385396e-01 7.31719227e-04
 3.75985161e-01 7.31719227e-04 1.11385396e-01 1.53413313e-03
 8.68784064e-02 2.63255204e-04 2.55626630e-03 5.21397327e-02
 3.27734757e-03 1.10627972e-02 2.48685358e-02 3.27734757e-03
 2.55626630e-03 8.94710875e-03 1.53413313e-03 7.31719227e-04
 2.63255204e-04]


In [35]:
# return b values, probability of drawing each, probability of moving from one beta state 
# to another (including staying at the current one)

def make_betas(beta_hi, dbeta, omega, q):
    """Return beta grid [beta_hi-dbeta, beta_high] and transition matrix,
    where q is probability of getting new random draw from [1-omega, omega]"""
    # q = prob of drawing new b
    # omega = prob of getting beta high
    beta_lo = beta_hi - dbeta
    # beta high = patient; beta low = impatient; dbeta = difference
    b_grid = np.array([beta_lo, beta_hi])
     #2-value array with each beta value
    pi_b = np.array([1 - omega, omega])
    #2-value array with probability of bhigh, blow if a random draw is needed
    Pi_b = (1-q)*np.eye(2) + q*np.outer(np.ones(2), pi_b)
    return b_grid, Pi_b, pi_b

`np.eye(2)` returns a 2x2 identity matrix => the matrix gets multiplied by the probability of not having to draw a new beta => on the diagonal, 0 otherwise

`np.ones(2)` returns a 1x2 "ones" matrix (vector)

`np.outer`computes the products of all values of two vectors (here of size 2 and 2) => returns a matrix of size (2,2) => technically duplicates pi_b into two rows => then multiplies all by q

`Pi_b` = diagonals: probs of staying at your current b; Other: probability of changing. First row: starting state = blow; starting state 0 bhigh.

In [36]:
def make_grids(min_a, max_a, n_a, beta_hi, dbeta, omega, q):
    ### asset and beta grids
    a_grid = sj.grids.asset_grid(min_a, max_a, n_a) # creates a grid for assets 
    #(probably tightly spaced for low values, the contrary for high values, as is common in HA models)

    b_grid_short, Pi_b, pi_b = make_betas(beta_hi, dbeta, omega, q) # calls the make_betas fct to get the grid of two beta values, 
    # probability of switching and transition matrix for beta

    ### combine grids for beta and e (latter pre-loaded from KMV above)
    e_grid = np.kron(np.ones_like(b_grid_short), e_grid_short)
    # repeats e_grid_short for each beta state?
    beta = np.kron(b_grid_short, np.ones_like(e_grid_short))
    # repeats b_grid_short for each e state?
    
    Pi = np.kron(Pi_b, Pi_e) 
    #combine transition matrices for joint beta and e states?
    pi_pdf = np.kron(pi_b, pi_e)
    # combined mass of agents in equilibrium for each joint value of beta and e?
    return e_grid, Pi, a_grid, beta, pi_pdf

`np.ones_like(b_grid_short)`= Returns an array of ones with the same shape and type as b_grid_short. (same for e_grid_short). 

Kronenberg product: "multiplies" each element of a matrix to the second  matrix. Denoted by ⊗. Used to combine independent Markov processes.

Returns:
* `a_grid` = asset grid ([Not evenly spaced but non linear and increasing](https://github.com/shade-econ/sequence-jacobian/blob/master/src/sequence_jacobian/utilities/discretize.py))
* `e_grid`= Full grid of productivity states, repeated for each discount factor state. (?)
* `beta` = Full grid of discount factors, repeated for each productivity state.
* `pi_pdf` = stationary distribution over all joint (β,e)(β,e)-states.
* `Pi`= transitions between all combinations of (β,e)(β,e)-states.

In [37]:
def income(wN_aftertax, N, e_grid, Tr_lumpsum, Tax_richest, zeta, pi_pdf):
        # after tax income, total L supply, all possible states of prod grid, lumpsum transfer, 
        # tax for the richest, cyclicality parameter (MP exercise page 17), stationary distrib 
        # over (share of agents in each) prod states

    # Auclert-Rognlie 2020 incidence function for labor income, with cyclicality parameter zeta
    # in default case with zeta = 0, this is just gamma / N = 1 and irrelevant
        
    gamma_N = e_grid ** (zeta * np.log(N)) / np.vdot(
        e_grid ** (zeta * np.log(N)), pi_pdf
    ) 

    # net after-tax income (include lump-sum transfer option)
    y = wN_aftertax * e_grid * gamma_N + Tr_lumpsum

    # also add option to tax richest type at margin
    y = y.reshape(-1, n_e)  # reshape to beta*e grid
    y[:, -1] -= Tax_richest / pi_e[-1].sum()  # tax richest e type
    y = y.ravel()  # flatten back
    return y

`gamma_N`= incidence fct in page 17. Reaxes the assumption that all benefit equally from the labor marjet changes that arise from a demand shock. (I didnt really understand but probably not necessary for our project).

`y`= disposable income. Seems to be the after tax income multiplied by the productivity state (ie the prod state enhances or reduces the originally same wage for everybody (given labor?)) multiplied by `gamma_N` (whatever it is it must be personal and enhance differences in wages across people) + transfers from the govt

`reshape` = y (vector) is reshaped into a 2D matrix with dimensions corresponding to the number of e states, each column an increasing level of productivity.

=> tax the richest by deducing a marginal tax to the richest (in terms of income!!!) agent type that is normalized by the sum of the stationary distribution of those values of e so that all agents receive a proportional decution

=> retransform y into a vector 

Return `y` = The vector of net after-tax incomes for all agents, with possible lump-sum transfers and tax on the richest agents.

In [38]:
@sj.het( #mark of an HA block
    exogenous="Pi", policy="a", backward="Va", backward_init=sj.hetblocks.hh_sim.hh_init
)
# define the exogenous variables: matrix of prod*beta states Pi, savings (as a policy function),
# backward-looking value function, and the solving method (backwards iteration?)
def hh_raw(Va_p, a_grid, y, r, beta, eis):
    """Household block. Slightly modify sequence_jacobian.hetblocks.hh_sim.hh to allow for beta vector"""
    # Inputs: derivative of value function Va_p, grid of asset values, a certain income (?) level, the hh's beta, 
    # and elasticity of intertemporal substitution (??)

    uc_nextgrid = (
        beta[:, np.newaxis] * Va_p
    )  # beta now vector, multiply Va_prime by row (e state)

    c_nextgrid = uc_nextgrid ** (-eis)
    coh = (1 + r) * a_grid[np.newaxis, :] + y[:, np.newaxis] 
    a = sj.interpolate.interpolate_y(c_nextgrid + a_grid, coh, a_grid)
    sj.misc.setmin(a, a_grid[0])
    c = coh - a
    Va = (1 + r) * c ** (-1 / eis)
    return Va, a, c

# returns The updated marginal value of assets for the current period, the policy function
# for savings, current-period consumption.

`coh`= cash on hands fct: (1+r)*a + y for all agents 

`sj.interpolate.interpolate_y`= 

TO BE FURTHER STUDIED


In [39]:
"""Consolidated HA household block and calibration of all exogenously-set parameters"""

hh_ha = hh_raw.add_hetinputs([make_grids, income])
hh_ha.name = "hh_ha"
calibration_ha = dict(
    eis=1,
    min_a=0,
    max_a=4000,
    n_a=200,
    r=0.005,
    q=0.01,
    Tr_lumpsum=0,
    Tax_richest=0,
    zeta=0,
)

# note that after-tax wages and labor are determined in equilibrium in full model, but we'll calibrate hh separately first
# steady-state normalized to Y = N = 1, out of which asset income on A = 20 at r = 0.005 is 0.1,
# and government spending is G=0.2, so markups + taxes total take 0.3
calibration_ha["N"], calibration_ha["wN_aftertax"] = 1, 0.7



In [40]:
"""Two-agent and rep-agent blocks"""


@sj.solved(unknowns={"C_RA": 1, "A": 1}, targets=["euler", "budget_constraint"])
def hh_ta(C_RA, A, wN_aftertax, eis, beta, r, lam):
    euler = (beta * (1 + r(+1))) ** (-eis) * C_RA(
        +1
    ) - C_RA  # Euler eq for consumption of infinitely lived household
    C_H2M = wN_aftertax  # consumption of hand to mouth household
    C = (1 - lam) * C_RA + lam * C_H2M
    budget_constraint = (
        (1 + r) * A(-1) + wN_aftertax - C - A
    )  # budget constraint for infinitely lived household
    return euler, budget_constraint, C_H2M, C


@sj.solved(unknowns={"C": 1, "A": 1}, targets=["euler", "budget_constraint"])
def hh_ra(C, A, wN_aftertax, eis, beta, r):
    euler = (beta * (1 + r(+1))) ** (-eis) * C(+1) - C
    budget_constraint = (1 + r) * A(-1) + wN_aftertax - C - A
    return euler, budget_constraint


# Understanding Calibration file

In [42]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import json

from household import (hh_ha as hh,
                       calibration_ha as calibration)

# for plotting:
plt.rcParams.update({'font.size': 10, 'font.family': 'serif', 'figure.figsize': (4, 3)})


In [ ]:
### Targets on the paper: